# Capstone Project: Battle of the Neighborhoods

## Directory

* [Introduction: The Problem](#'intro')
* [Data](#data)
* [Analysis](#analysis)
* [Reframing the Question](#frame)
* [Inferences](#inference)

### Introduction: The Problem <a name= 'intro'></a>

In this problem, a group of angel investors has approached my analysis firm with a question. They want to invest in trending buisnesses in the areas of Miami Beach, Florida. They are paricularly interested in non-chain coffeehouses and local breweries/distilleries. They want to the firm to divide the areas of Miami Beach into neighborhoods, and then inform them of the trending popularity of coffeehouses, breweries, and distilleries to see if it is worth the price to invest in Miami Beach.

### Data  <a name= 'data'></a>

Original data on the individual neighborhoods were obtained from Wikipeida. Latitude and Longitude were manually entered into a CSV file. The Foursquare API was used to generate the trending data. 

In [1]:
import pandas as pd
coords = pd.read_csv(r'C:\Users\emcollin\OneDrive - Volusia County Schools\Documents\Projects\IBM Data Science Final\miami_coords.csv')
coords.head()

,Neighborhood,Latitude,Longitude
0,Belle Isle,25.7913,80.1477
1,City Center,25.7617,80.1918
2,Di Lido Island,25.7906,80.1589
3,Flagler Monument Island,25.7853,80.1528
4,Flamingo/Lumus,25.7794,80.1351


The Longitudes should be negetive, not positive. Let's correct that.

In [2]:
coords['Longitude'] *= -1
coords.head()

,Neighborhood,Latitude,Longitude
0,Belle Isle,25.7913,-80.1477
1,City Center,25.7617,-80.1918
2,Di Lido Island,25.7906,-80.1589
3,Flagler Monument Island,25.7853,-80.1528
4,Flamingo/Lumus,25.7794,-80.1351


In [3]:
coords.set_index('Neighborhood', inplace = True)

In [4]:
coords.shape

(17, 2)

### Analysis <a name= 'analysis'></a>

At the beginning of any analysis, it's a good idea to visualize data. Lets show where exactly the neighborhoods are in Miami using Folium

In [5]:
import folium

In [6]:
miami_map = folium.Map(location=[25.7907 , -80.1301],zoom_start=12)

for lat,lng,neighbourhood in zip(coords['Latitude'],coords['Longitude'],coords.index):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(miami_map)
miami_map

Fantastic! We can easily see how the city is divided into it's 17 neighborhoods. Now let's start with some API Calls. We can use the FourSquare API to check for trending businesses in the area.

Let's start with just one area, Belle Island. 

In [7]:
import requests

In [8]:
CLIENT_ID = REDACTED
CLIENT_SECRET = REDACTED
ACCESS_TOKEN = REDACTED
VERSION = '20180604'
LIMIT = 30
RADIUS = 500

In [9]:
lat = coords.loc['Belle Isle', 'Latitude']
long = coords.loc['Belle Isle', 'Longitude']

In [10]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&oauth_token={}&limit={}&radius={}&v={}'.format(CLIENT_ID, CLIENT_SECRET,lat,long,ACCESS_TOKEN,LIMIT,RADIUS,VERSION)

In [11]:
url

'https://api.foursquare.com/v2/venues/trending?client_id=G0LEPQWK5EK5SAOVKY2FHY2CYWW4PCVAPTLZEDXUERAJMWEW&client_secret=NHA4RLDNRQYGGCGOHWT3MVLYMEK4IEIGFDHLYMCEQWNFIPQA&ll=25.7913,-80.1477&oauth_token=5WOW1XMUB50CDU0JUPIEEZA5U4UG1GS0LJZFTXKZUGBWTQTA&limit=30&radius=500&v=20180604'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b6734be8de2614ce304d79'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

Weird, there's no trending venues within 100 meteres of Belle Isle. Lets look a little closer at the map

In [13]:
miami_map = folium.Map(location=[lat,long],zoom_start=17)

for lat,lng,neighbourhood in zip(coords['Latitude'],coords['Longitude'],coords.index):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(miami_map)
miami_map

Belle Isle is a real Island! There's hardly any shopping on it, and it's surrounded by water. Let's start collecting data for the other neighborhoods, and maybe increase the range a little. 

In [14]:
lat = coords.loc['City Center', 'Latitude']
long = coords.loc['City Center', 'Longitude']
RADIUS = 500

In [15]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&oauth_token={}&limit={}&radius={}&v={}'.format(CLIENT_ID, CLIENT_SECRET,lat,long,ACCESS_TOKEN,LIMIT,RADIUS,VERSION)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60b6734b40b564286060a89e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

There's no results for trending venues at the City Center with a radius of half a kilometer. Something is off. Let's look at the map again

In [16]:
miami_map = folium.Map(location=[lat, long],zoom_start=15)

for lat,lng,neighbourhood in zip(coords['Latitude'],coords['Longitude'],coords.index):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(miami_map)
miami_map

We should absolutley be getting some trending stores or restaraunts, but we aren't. Let's switch tact.

## Reframing the Question  <a name= 'frame'></a>

Instead of looking at trending stores, let's look at the total count of coffee shops, breweries, and distilleries in the individual neighborhoods, and draw conclusions from that.  

In [17]:
RADIUS = 1000

In [18]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&limit={}&radius={}&categoryId=4bf58dd8d48988d1e0931735,50327c8591d4c4b30a586d5d,4e0e22f5a56208c4ea9a85a0&v={}'.format(CLIENT_ID, CLIENT_SECRET,lat,long,ACCESS_TOKEN,LIMIT,RADIUS,VERSION)

In [19]:
url

'https://api.foursquare.com/v2/venues/search?client_id=G0LEPQWK5EK5SAOVKY2FHY2CYWW4PCVAPTLZEDXUERAJMWEW&client_secret=NHA4RLDNRQYGGCGOHWT3MVLYMEK4IEIGFDHLYMCEQWNFIPQA&ll=25.9331,-80.1918&oauth_token=5WOW1XMUB50CDU0JUPIEEZA5U4UG1GS0LJZFTXKZUGBWTQTA&limit=30&radius=1000&categoryId=4bf58dd8d48988d1e0931735,50327c8591d4c4b30a586d5d,4e0e22f5a56208c4ea9a85a0&v=20180604'

In [20]:
counts = pd.DataFrame(columns = ['Neighborhood'])
for index, row in coords.iterrows():
    lat = coords.loc[index, 'Latitude']
    long = coords.loc[index, 'Longitude']
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&limit={}&radius={}&categoryId=4bf58dd8d48988d1e0931735,50327c8591d4c4b30a586d5d,4e0e22f5a56208c4ea9a85a0&v={}'.format(CLIENT_ID, CLIENT_SECRET,lat,long,ACCESS_TOKEN,LIMIT,RADIUS,VERSION)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if len(venues) > 0:
        dataframe = pd.json_normalize(venues)
        dataframe['Neighborhood'] = index
        counts = counts.append(dataframe)

In [21]:
counts

,Neighborhood,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.crossStreet,venuePage.id
0,Belle Isle,598762f90d8a0f59004f42a6,Dr Smood,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",v-1622569804,False,1800 Bay Rd Ste 103,25.793790,-80.143940,"[{'label': 'display', 'lat': 25.79379, 'lng': ...",...,United States,"[1800 Bay Rd Ste 103, Miami Beach, FL 33139]",2271609,https://www.grubhub.com/restaurant/dr-smood-18...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
1,Belle Isle,5190c8e1498eb00f7f23421f,Panther Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622569804,False,1875 Purdy Ave,25.794516,-80.144275,"[{'label': 'display', 'lat': 25.79451570978172...",...,United States,"[1875 Purdy Ave, Miami Beach, FL 33139]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Belle Isle,5bd755f49e0d54002c26cb98,Delicious Raw,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622569804,False,Bay Rd,25.794059,-80.143829,"[{'label': 'display', 'lat': 25.794059, 'lng':...",...,United States,"[Bay Rd, Miami Beach, FL 33139]",1626483,https://www.grubhub.com/restaurant/delicious-r...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
3,Belle Isle,5ad4eb8ccf72a02b59a09fc0,Peet's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622569804,False,NaN,25.790604,-80.139350,"[{'label': 'display', 'lat': 25.790604, 'lng':...",...,United States,"[Miami Beach, FL 33139]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Belle Isle,4f32868b19836c91c7e0593b,Tea For Two,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622569804,False,1205 17th St,25.792325,-80.141600,"[{'label': 'display', 'lat': 25.792325, 'lng':...",...,United States,"[1205 17th St, Miami Beach, FL 33139]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Nautilus,5693d71c498ee1905925eaa9,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622569806,False,827 West 41st St,25.813974,-80.134140,"[{'label': 'display', 'lat': 25.81397438, 'lng...",...,United States,"[827 West 41st St (at N Jefferson Ave), Miami ...",NaN,NaN,NaN,NaN,NaN,NaN,at N Jefferson Ave,NaN
1,Nautilus,51c849d5498ea9955fd9bee3,Coffee Place At Mount Sinai Medical Building 4308,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",v-1622569806,False,4308 Alton Rd,25.814460,-80.141002,"[{'label': 'display', 'lat': 25.81446028796437...",...,United States,"[4308 Alton Rd, Miami Beach, FL 33140]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nautilus,4b472156f964a520bf2b26e3,Dunkin',"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",v-1622569806,False,341 W 41st St,25.814040,-80.127333,"[{'label': 'display', 'lat': 25.81403962645725...",...,United States,"[341 W 41st St (Sheridan Ave), Miami Beach, FL...",2133270,https://www.grubhub.com/restaurant/dunkin-341-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,Sheridan Ave,453290138
0,Isle of Normandy,4d99256f744f370482490158,Dolce Vita Gelato Cafe,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",v-1622569806,False,954 Normandy Dr,25.855648,-80.129175,"[{'label': 'display', 'lat': 25.85564838300050...",...,United States,"[954 Normandy Dr, Miami Beach, FL 33141]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


That's pretty ugly. We just want the neighborhood, the name of the establishment, and the category type. Let's clean it up.

In [22]:
counts_modified = counts[['Neighborhood', 'name', 'categories']]

In [23]:
counts_modified.shape

(132, 3)

In [24]:
pd.set_option("display.max_rows", 15, "display.max_columns", None)
counts_modified

,Neighborhood,name,categories
0,Belle Isle,Dr Smood,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J..."
1,Belle Isle,Panther Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
2,Belle Isle,Delicious Raw,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
3,Belle Isle,Peet's Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
4,Belle Isle,Tea For Two,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
...,...,...,...
0,Nautilus,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
1,Nautilus,Coffee Place At Mount Sinai Medical Building 4308,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
2,Nautilus,Dunkin',"[{'id': '4bf58dd8d48988d148941735', 'name': 'D..."
0,Isle of Normandy,Dolce Vita Gelato Cafe,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I..."


That's better but let's split the category id and name.

In [25]:
final_count = counts_modified.copy()
ID = list()
name = list()

for index, value in final_count['categories'].items():
    ID.append(counts['categories'].iloc[index][0]['id'])
    name.append(counts['categories'].iloc[index][0]['name']) 

final_count['cat_id'] = ID
final_count['cat_name'] = name

In [26]:
final_count = final_count.drop('categories', axis = 1)

In [27]:
final_count.set_index('Neighborhood', inplace = True)

In [28]:
final_count

,name,cat_id,cat_name
Neighborhood,,,
Belle Isle,Dr Smood,4bf58dd8d48988d112941735,Juice Bar
Belle Isle,Panther Coffee,4bf58dd8d48988d1e0931735,Coffee Shop
Belle Isle,Delicious Raw,4bf58dd8d48988d1e0931735,Coffee Shop
Belle Isle,Peet's Coffee,4bf58dd8d48988d1e0931735,Coffee Shop
Belle Isle,Tea For Two,4bf58dd8d48988d1e0931735,Coffee Shop
...,...,...,...
Nautilus,Starbucks,4bf58dd8d48988d112941735,Juice Bar
Nautilus,Coffee Place At Mount Sinai Medical Building 4308,4bf58dd8d48988d1e0931735,Coffee Shop
Nautilus,Dunkin',4bf58dd8d48988d1e0931735,Coffee Shop


In [29]:
final_count['cat_name'].unique()

array(['Juice Bar', 'Coffee Shop', 'Café', 'Bakery', 'Brewery', 'Bar',
       'Donut Shop', 'Bagel Shop'], dtype=object)

And now we see we have 8 different kinds of categories for stores within a kilometer of the center of each neighborhood. 

Let's take a look at the counts of each different store. 

In [30]:
final_count['cat_name'].value_counts()

Coffee Shop    85
Juice Bar      19
Brewery         6
Café            6
Bakery          4
Donut Shop      4
Bar             4
Bagel Shop      4
Name: cat_name, dtype: int64

The vast majority are coffee shops. What about the names of the businesses?

In [31]:
final_count['name'].value_counts()

Starbucks                        26
Dunkin'                           5
JOE & THE JUICE                   5
Panther Coffee                    4
Einstein Bros Bagels              3
                                 ..
Starbucks 13th street             1
Dos Cafes                         1
Les Moulins La Fayette            1
Nespresso Boutique Bar, Miami     1
Dolce Vita Gelato Cafe            1
Name: name, Length: 66, dtype: int64

Lets take a look by neighborhood.

In [32]:
by_neighborhood = final_count.reset_index()
by_neighborhood

,Neighborhood,name,cat_id,cat_name
0,Belle Isle,Dr Smood,4bf58dd8d48988d112941735,Juice Bar
1,Belle Isle,Panther Coffee,4bf58dd8d48988d1e0931735,Coffee Shop
2,Belle Isle,Delicious Raw,4bf58dd8d48988d1e0931735,Coffee Shop
3,Belle Isle,Peet's Coffee,4bf58dd8d48988d1e0931735,Coffee Shop
4,Belle Isle,Tea For Two,4bf58dd8d48988d1e0931735,Coffee Shop
...,...,...,...,...
127,Nautilus,Starbucks,4bf58dd8d48988d112941735,Juice Bar
128,Nautilus,Coffee Place At Mount Sinai Medical Building 4308,4bf58dd8d48988d1e0931735,Coffee Shop
129,Nautilus,Dunkin',4bf58dd8d48988d1e0931735,Coffee Shop
130,Isle of Normandy,Dolce Vita Gelato Cafe,4bf58dd8d48988d112941735,Juice Bar


In [33]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
by_neighborhood.groupby(['Neighborhood', 'name'])['cat_name'].count()

Neighborhood             name                                             
Bayshore                 Brothers Cafe & Market                               1
                         Capital Take Away                                    1
                         Dunkin'                                              1
                         Fresco Cafe                                          1
                         Nativ Made                                           1
                         Neighbor                                             1
                         Shepherd Artisan Coffee                              1
                         Starbucks                                            4
Belle Isle               Abbey Brewing Company                                1
                         CAO Bakery & Cafe                                    1
                         Delicious Raw                                        1
                         Dr Smood            

Let's take a look at types of places

In [34]:
by_neighborhood.groupby(['Neighborhood', 'cat_name'])['name'].count()

Neighborhood             cat_name   
Bayshore                 Café            1
                         Coffee Shop     8
                         Juice Bar       2
Belle Isle               Bagel Shop      1
                         Bakery          1
                         Bar             1
                         Brewery         1
                         Café            1
                         Coffee Shop     9
                         Donut Shop      1
                         Juice Bar       2
City Center              Bagel Shop      1
                         Bakery          1
                         Bar             1
                         Brewery         1
                         Café            1
                         Coffee Shop    15
                         Donut Shop      1
                         Juice Bar       2
Flagler Monument Island  Juice Bar       1
Flamingo/Lumus           Bagel Shop      1
                         Bakery          1
                 

And as a final bit of analysis, let's plot the individual shops

In [35]:
lat = coords.loc['Belle Isle', 'Latitude']
long = coords.loc['Belle Isle', 'Longitude']

In [36]:
miami_map = folium.Map(location=[lat, long],zoom_start=13)

for lat,lng,neighbourhood in zip(counts['location.lat'],counts['location.lng'],counts['Neighborhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(miami_map)
miami_map

## Inferences  <a name= 'inference'></a>

* There are many, many more Starbucks than any other kind of business
* There are places where additional coffee shops may be of help, but there are no breweries or distilleries in the entire city
* There are no places to get coffee on Bella Isle

### Recommendation

In total, the current underserved crowd is the craft-beer and small-batch distillery crowd. Opening such a bar or place would generate business. If you are most interested in securing income, but least interested in diversifying the city, open a Starbucks. Otherwise, invest in a local brewery or distillery. If you chose to open a ANY coffee shop, put it on Bella Island.